In [2]:
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings("ignore")

In [3]:
user_ids = np.array([0, 1, 2, 3, 4, 0, 1, 2, 3, 4])
item_ids = np.array([0, 1, 2, 3, 4, 1, 2, 3, 4, 5])
ratings  = np.array([5, 4, 3, 2, 1, 4, 5, 3, 2, 1])

In [4]:
user_ids_train, user_ids_test, item_ids_train, item_ids_test, ratings_train, ratings_test = train_test_split(user_ids, item_ids, ratings, test_size = 0.2, random_state = 42)

In [ ]:
def create_model(num_users, num_items, embedding_dim):
    user_input = Input(shape = (1,), name = "user")
    item_input = Input(shape = (1,), name = "item")

    user_embedding = Embedding(input_dim=num_users, output_dim=embedding_dim, name="user_embedding")(user_input) # shape -> (batch_size, user_input, embedding_dim)
    item_embedding = Embedding(input_dim=num_items, output_dim=embedding_dim, name="item_embedding")(item_input) # shape -> (batch_size, user_input, embedding_dim)

    user_vecs = Flatten()(user_embedding) # shape -> (batch_size, embedding_dim)
    item_vecs = Flatten()(item_embedding) # shape -> (batch_size, embedding_dim)

    '''
        user_vecs.shape = (1,8) olsun  [[0.1, 0.2, ....]] (batch_size, embedding_dim)
        item_vecs.shape = (1,8) olsun  [[0.8, 0.6, ....]] (batch_size, embedding_dim)
        axes = 1 -> embedding vektörlerini çarp sonra üm değerleri topla
    '''
    dot_product = Dot(axes = 1)([user_vecs, item_vecs]) # shape -> (1, 1) -> (batch_size, sum_result)
    output = Dense(1)(dot_product)                      # Dense -> fully connected layer (Linear) output = weight * dot_product + bias

    model = Model(inputs = [user_input, item_input], outputs = output)
    model.compile(optimizer = Adam(learning_rate=0.01), loss = "mean_squared_error")

    return model

In [9]:
num_users = 5
num_items = 6
embedding_dim = 8
model = create_model(num_users, num_items, embedding_dim)
model.fit([user_ids_train, item_ids_train], ratings_train, epochs = 10, verbose = 1, validation_split = 0.1)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 822ms/step - loss: 9.2854 - val_loss: 24.8444
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 9.2049 - val_loss: 24.7583
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 9.1213 - val_loss: 24.6718
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 9.0316 - val_loss: 24.5850
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 8.9334 - val_loss: 24.4980
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 8.8246 - val_loss: 24.4107
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 8.7037 - val_loss: 24.3230
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 8.5699 - val_loss: 24.2349
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 8.4225 - val_loss: 24.1466
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 8.2611 - val_loss: 24.0582


In [ ]:
import torch
import torch.nn as nn

class CreateModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super().__init__()
        self.embeddding_1 = nn.Embedding(num_embeddings = num_users, embedding_dim = embedding_dim)
        self.embeddding_2 = nn.Embedding(num_embeddings = num_items, embedding_dim = embedding_dim)
        self.output_layer = nn.Linear(1, 1)

    def forward(self, user_input, item_input):
        emb_1 = self.embeddding_1(user_input)
        emb_2 = self.embeddding_2(item_input)
        dot = torch.sum(emb_1*emb_2, dim = 1, keepdim=True)
        return self.output_layer(dot)